<a href="https://colab.research.google.com/github/sugarforever/huggingface-tutorials/blob/main/hg_03_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 小白入门HuggingFace

## 03 Transformers的神器 - Pipeline

`Pipeline` 特性极度简化模型使用过程，使得从Hub上使用任何模型进行各种任务变得有趣而简单。即使没有特定模态经验，或者不熟悉模型背后逻辑的小白，仍然可以通过 `Pipeline` 进行推理!

`Transformers` 为 `Pipeline` 特性提供了接口 `pipeline()` 函数。

我们在本期会学习如何使用 `pipeline()` 函数。



## 准备Python环境

In [1]:
!pip install git+https://github.com/huggingface/transformers sentencepiece --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00


## 文本摘要生成的例子

每个任务类型都有相应的Pipeline类，但使用通用的 `pipeline()` 函数更简单，该函数会自动加载默认模型，或指定模型，以及能够对您的任务进行推理的预处理类。


In [ ]:
from transformers import pipeline

generator = pipeline("summarization")
print(generator.__class__)

In [12]:
from transformers import pipeline

generator = pipeline(model="knkarthick/MEETING_SUMMARY")
print(generator.__class__)

<class 'transformers.pipelines.text2text_generation.SummarizationPipeline'>


In [13]:
text = """
London, the capital of England and the United Kingdom, is a 21st-century city with history stretching back to Roman times. At its centre stand the imposing Houses of Parliament, the iconic ‘Big Ben’ clock tower and Westminster Abbey, site of British monarch coronations. Across the Thames River, the London Eye observation wheel provides panoramic views of the South Bank cultural complex, and the entire city.
"""
generator(text)

[{'summary_text': 'The London Eye offers panoramic views of the South Bank and the entire city.'}]

## 图像识别的例子

In [14]:
from transformers import pipeline

classifier = pipeline(model="google/vit-base-patch16-224")
print(classifier.__class__)

<class 'transformers.pipelines.image_classification.ImageClassificationPipeline'>


In [15]:
image_url = "https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png"
classifier(image_url)

[{'score': 0.9905232787132263, 'label': 'macaw'},
 {'score': 0.005603468045592308,
  'label': 'African grey, African gray, Psittacus erithacus'},
 {'score': 0.0010569051373749971, 'label': 'toucan'},
 {'score': 0.0006811480852775276,
  'label': 'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita'},
 {'score': 0.0006714325281791389, 'label': 'lorikeet'}]

## pipeline函数的参数

`pipeline()` 函数除了 `task`, `model` 参数，还支持一系列模型通用的参数，和模型特定的参数。

介绍几个模型通用的重要的参数：

1. `device`

    device参数接受以下几种可能的值:
    - cpu：将模型加载到CPU上进行推理。
    - cuda：将模型加载到第一块GPU进行推理，等同于'cuda:0'。
    - cuda:X：将模型加载到编号为X的GPU上进行推理，X是GPU编号。
    - mps：将模型加载到Mac OS上的Apple M1或M2芯片上利用Metal Performance Shaders (MPS) 进行推理。
    - tpu：将模型加载到Tensor Processing Unit (TPU) 上进行推理,如Google Colab中的TPU。
    - auto：自动选择一个可用的设备，会按优先级首先选择TPU，然后是GPU，最后是CPU。

    在huggingface transformers的pipeline函数中，device参数也可以设置为一个整数，比如0，是为了将模型加载到特定编号的 `GPU` 进行推理。

2. `batch_size`

    默认情况下，pipeline并不会做批量推理（batch inference）。但是，pipeline函数依然提供了 `batch_size` 参数支持必要的batch inference。
    
    `batch_size` 参数会影响到:
    - 推理速度：增加batch_size通常可以提高GPU利用效率，加速推理速度。但若过大也会导致OOM。
    - 内存占用：较大的batch_size会占用更多GPU显存。
    - 推理效果：某些模型对batch_size比较敏感,大小不同会影响准确率。
    - 支持长度：一些模型对最大输入长度有限制,batch_size过大可能超出模型支持长度。
    - 输出：输出会是一个批量结果列表,如果只需要单条结果,需要索引获取。

    所以设置合适的batch_size需要综合考虑这些因素。


### 模型特有的参数

我们以文本分类为例，介绍模型特有参数的使用。

文本分类的pipeline，由类 `TextClassificationPipeline` 实现，请参考文档 [TextClassificationPipeline](https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/pipelines#transformers.TextClassificationPipeline)

该类提供了模型特有的参数 `return_all_scores`，它用来标识是否返回所有预测分数或者只返回预测命中类别的分数。

示例如下：

In [16]:
from transformers import pipeline
classifier = pipeline(model="ProsusAI/finbert")
print(classifier.__class__)

<class 'transformers.pipelines.text_classification.TextClassificationPipeline'>


In [17]:
sentence = """
Hundreds of new North Sea oil and gas licences to boost British energy independence and grow the economy
"""

In [18]:
classifier(sentence)

[{'label': 'positive', 'score': 0.886681854724884}]

In [19]:
classifier(sentence, return_all_scores=True)

[[{'label': 'positive', 'score': 0.886681854724884},
  {'label': 'negative', 'score': 0.009547530673444271},
  {'label': 'neutral', 'score': 0.10377060621976852}]]